# Housing Prices: Advanced Regression Techniques

## 1. Business Understanding

## 2. Data Understanding

In [180]:
# Import packages 
import pandas as pd
import numpy as np
from sklearn.preprocessing import FunctionTransformer, StandardScaler, Imputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.feature_extraction import FeatureHasher
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.pyplot as plt
import math
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBClassifier, XGBRegressor

import pandas_profiling
from sklearn.ensemble import RandomForestRegressor

In [3]:
# Load the data
full_train = pd.read_csv('train.csv', squeeze=True)
test = pd.read_csv('test.csv', squeeze=True)
sample_submission = pd.read_csv('sample_submission.csv', squeeze=True)

#### Dataset Description/EDA

What does each row represent? What are the columns?

Distributions of features/target, missing values, relationships between features etc.

In [4]:
print(full_train.shape)
print(test.shape)

(1460, 81)
(1459, 80)


In [69]:
# Storing categorical and numerical fields 
full_s_fields = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 
                 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 
                 'BldgType', 'HouseStyle', 'OverallQual', 'RoofStyle', 'RoofMatl', 'Exterior1st', 
                 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 
                 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 
                 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 
                 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 
                 'MiscFeature', 'SaleType', 'MSSubClass', 'SaleCondition']

full_n_fields = ['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 
                 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 
                 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 
                 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 
                 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'YrSold', 'MoSold', 
                 'MiscVal','OverallCond']

In [6]:
pandas_profiling.ProfileReport(full_train)

Number of variables,81
Number of observations,1460
Total Missing (%),5.9%
Total size in memory,924.0 KiB
Average record size in memory,648.1 B
Numeric,38
Categorical,43
Boolean,0
Date,0
Text (Unique),0
Rejected,0


#### Missing Values

Below we investigate the percent of missing values per column. There are 5 columns with more than 20% of the values missing. We decide to exclude these 5 columns (PoolQC, MiscFeature, Alley, Fence, FireplaceQu).

In [70]:
# Find fraction of missing values per column
frac_missing = full_train.apply(lambda col: col.isnull()).sum().sort_values(ascending=False)/full_train.shape[0]
frac_missing.head(10)

PoolQC          0.995205
MiscFeature     0.963014
Alley           0.937671
Fence           0.807534
FireplaceQu     0.472603
LotFrontage     0.177397
GarageCond      0.055479
GarageType      0.055479
GarageYrBlt     0.055479
GarageFinish    0.055479
dtype: float64

In [71]:
# Store the features that will be excluded
missing_excl = list(frac_missing[frac_missing>.2].index)
missing_excl

['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu']

#### Columns with high percentage of Zero Values

In [72]:
# Find fraction of zero values per column
frac_zeros = full_train.apply(lambda col: col == 0).sum().sort_values(ascending=False)/full_train.shape[0]
frac_zeros.head(10)

PoolArea         0.995205
3SsnPorch        0.983562
LowQualFinSF     0.982192
MiscVal          0.964384
BsmtHalfBath     0.943836
ScreenPorch      0.920548
BsmtFinSF2       0.885616
EnclosedPorch    0.857534
HalfBath         0.625342
MasVnrArea       0.589726
dtype: float64

In [73]:
# Store the features that will be excluded
zeros_excl = list(frac_zeros[frac_zeros>.9].index)
zeros_excl

['PoolArea',
 '3SsnPorch',
 'LowQualFinSF',
 'MiscVal',
 'BsmtHalfBath',
 'ScreenPorch']

#### Categorical variables with highly unbalanced classes

In [74]:
frac_cat = full_train[full_s_fields].apply(lambda col: col.value_counts().max()).sort_values(ascending=False)/full_train.shape[0]
frac_cat.head(10)

Utilities     0.999315
Street        0.995890
Condition2    0.989726
RoofMatl      0.982192
Heating       0.978082
LandSlope     0.946575
CentralAir    0.934932
Functional    0.931507
PavedDrive    0.917808
Electrical    0.913699
dtype: float64

In [75]:
# Store the features that will be excluded
cat_excl = list(frac_cat[frac_cat>.9].index)
cat_excl

['Utilities',
 'Street',
 'Condition2',
 'RoofMatl',
 'Heating',
 'LandSlope',
 'CentralAir',
 'Functional',
 'PavedDrive',
 'Electrical',
 'GarageCond']

#### Top/Bottom Correlations

In [76]:
# Highest Correlation with SalePrice (Absolute Value)
full_train.corr()['SalePrice'].abs().sort_values(ascending=False).head(10)

SalePrice       1.000000
OverallQual     0.790982
GrLivArea       0.708624
GarageCars      0.640409
GarageArea      0.623431
TotalBsmtSF     0.613581
1stFlrSF        0.605852
FullBath        0.560664
TotRmsAbvGrd    0.533723
YearBuilt       0.522897
Name: SalePrice, dtype: float64

In [77]:
# Lowest Correlation with SalePrice (Absolute Value)
full_train.corr()['SalePrice'].abs().sort_values(ascending=True).head(10)

BsmtFinSF2      0.011378
BsmtHalfBath    0.016844
MiscVal         0.021190
Id              0.021917
LowQualFinSF    0.025606
YrSold          0.028923
3SsnPorch       0.044584
MoSold          0.046432
OverallCond     0.077856
MSSubClass      0.084284
Name: SalePrice, dtype: float64

## 3. Data Preparation

#### Excluding Columns based on EDA

In [78]:
# excluding columns 
s_fields = [c for c in full_s_fields if not c in missing_excl and not c in zeros_excl and not c in cat_excl]
n_fields = [c for c in full_n_fields if not c in missing_excl and not c in zeros_excl]

In [79]:
len(full_s_fields) + len(full_n_fields)

79

In [80]:
# Which out of the lowest correlated variables are still in the dataset 
low_corr = list(full_train.corr()['SalePrice'].abs().sort_values(ascending=True).head(10).index)
low_corr
for i in low_corr: 
    if i in s_fields: 
        print(i)
    elif i in n_fields: 
        print(i)

BsmtFinSF2
YrSold
MoSold
OverallCond
MSSubClass


#### Excluding Rows that contain outliers 

In [81]:
# Remove outliers greater than 4 standard deviations
new_train = full_train.copy()
to_drop = []
for r_idx, row in full_train.iterrows():
    for c_idx, val in enumerate(row):
        c_name = full_train.columns[c_idx]
        if c_name in n_fields:
            c_mean = full_train[c_name].mean()
            val_diff = np.abs(val - c_mean)
            c_std = full_train[c_name].std()
            if val_diff >= 4 * c_std:
                to_drop.append(r_idx)

train = new_train.drop(new_train.index[to_drop])
train.shape

(1306, 81)

#### Defining Target and Predictors

In [83]:
# Set target and predictors
target = 'SalePrice'
predictors = [c for c in train.columns if not c == target and not c == 'Id' 
              and not c in missing_excl and not c in zeros_excl and not c in cat_excl]

#### Data Processing Pipeline

In [227]:
def ensure_data_type(X):
    return X.apply(lambda col: col.astype(str))

def make_dictionaries(X):
    return X.to_dict(orient='records')

def select_categorical_features(X):
    return X[s_fields]

def select_numeric_features(X):
    return X[n_fields]

# Pipeline for numeric features 
numeric_feature_pipeline = Pipeline(steps=[
    ('select', FunctionTransformer(select_numeric_features, validate=False)),
    ('impute', Imputer(strategy='mean')), #'most_frequent'
    ('scale', StandardScaler())
])

# Pipeline for categorical features 
categorical_pipeline = Pipeline(steps=[
    ('select', FunctionTransformer(select_categorical_features, validate=False)),
    ('datatype', FunctionTransformer(ensure_data_type, validate=False)),
    ('dictionaries', FunctionTransformer(make_dictionaries, validate=False)),
    ('vectorize', DictVectorizer(sparse=False)) # 2000  FeatureHasher(n_features=10)   DictVectorizer(sparse=False)
])

fu = FeatureUnion([
    ('categorical', categorical_pipeline),
    ('numeric', numeric_feature_pipeline),
])

fr = Pipeline([
    ('union', fu),
    ('reduction', PCA(n_components=200) ),  #  PCA(n_components=200)   TruncatedSVD(n_components=400)
])

In [151]:
transformed_train =fu.fit_transform(X_train)
transformed_train.shape
#X_train.shape
#len(n_fields)

(979, 241)

## 4. Modeling

#### Train/Test Split

In [146]:
# Train/test split
X_train_full = train[predictors]
y_train_full = train[[target]]
X_train, X_dev, y_train, y_dev = train_test_split(X_train_full, y_train_full, random_state=1)
X_test = test[predictors]

#### Usefull Functions

In [96]:
# Write a prediction into a csv 
def make_csv(name, pred):
    new_df = test.copy()
    new_df['SalePrice'] = pred
    new_df[['Id', 'SalePrice']].to_csv(name, index=False)
        
def root_mean_square_error(pred, actual):
    log_actual = np.log(actual)
    log_pred = np.log(pred)
    return np.sqrt(mean_squared_error(log_actual, log_pred))

#### Ridge Regression

In [243]:
# Ridge Log Sales Price
model_pipe = Pipeline(steps=[
    ('preprocess', fr),
    ('predict', Ridge(alpha=10, fit_intercept=True, solver="cholesky"))
])

round_num = 2
model_pipe.fit(X_train, np.log(y_train).round(round_num))
print(model_pipe.score(X_dev, np.log(y_dev).round(round_num)))
print(root_mean_square_error(model_pipe.predict(X_dev), np.log(y_dev).round(round_num)))

0.9081798813800992
0.010073466704297722


In [244]:
model_pipe.fit(X_train_full, np.log(y_train_full))
rid_pred = model_pipe.predict(X_test)

In [ ]:
make_csv('logPrice_ridgeCP2.csv', np.exp(rid_pred))

In [94]:
# Gridsearch CV 
search_params = {
    'preprocess__union__categorical__vectorize__n_features': [10, 20, 38],
    'preprocess__union__numeric__impute__strategy': ['mean', 'median', 'most_frequent'],
    'preprocess__reduction__n_components': [2, 10, 15, 20, 35],
    'predict__alpha': [1, 10, 50, 100, 200, 500],
    'predict__fit_intercept': [True, False],
    'predict__solver': ["cholesky"],
    'predict__random_state': [1],
}

round_num = 2
grid_search = GridSearchCV(model_pipe, search_params, cv=5)
grid_search.fit(X_train, np.log(y_train).round(round_num))
print(grid_search.best_params_)
print(grid_search.score(X_dev, np.log(y_dev).round(round_num)))
print(root_mean_square_error(grid_search.predict(X_dev), np.log(y_dev).round(round_num)))

{'predict__alpha': 10, 'predict__fit_intercept': True, 'predict__random_state': 1, 'predict__solver': 'cholesky', 'preprocess__reduction__n_components': 35, 'preprocess__union__categorical__vectorize__n_features': 10, 'preprocess__union__numeric__impute__strategy': 'most_frequent'}
0.8838764284760346


NameError: name 'root_mean_square_error' is not defined

#### Random Forest

In [213]:
# Random Forest Log Sales Price
model_pipe = Pipeline(steps=[
    ('preprocess', fr),
    ('predict', RandomForestRegressor(n_estimators=200, oob_score=True))
])

round_num = 2
model_pipe.fit(X_train, np.log(y_train).round(round_num))
print(model_pipe.score(X_dev, np.log(y_dev).round(round_num)))
print(root_mean_square_error(model_pipe.predict(X_dev), np.log(y_dev).round(round_num)))

/Users/christina/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:270: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)


0.8760513269127863
0.011598566818177882


In [214]:
model_pipe.fit(X_train_full, np.log(y_train_full))
rf_pred = model_pipe.predict(X_test)
make_csv('logPrice_RFCP4.csv', np.exp(rf_pred))  #np.exp

/Users/christina/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:270: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)


In [195]:
# Gridsearch CV 
model_pipe = Pipeline(steps=[
    ('preprocess', fu),
    ('predict', RandomForestRegressor(n_estimators=100, oob_score=True))
])

search_params = {
  #  'preprocess__union__numeric__impute__strategy': ['mean', 'median', 'most_frequent'],
    'predict__n_estimators': [50,100,200,300,500],
    'predict__oob_score': [True,False],
}

round_num = 2
grid_search = GridSearchCV(model_pipe, search_params, cv=5)
grid_search.fit(X_train, np.log(y_train).round(round_num))
print(grid_search.best_params_)
print(grid_search.score(X_dev, np.log(y_dev).round(round_num)))
print(root_mean_square_error(grid_search.predict(X_dev), np.log(y_dev).round(round_num)))

/Users/christina/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:270: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/Users/christina/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:270: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/Users/christina/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:270: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/Users/christina/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:270: DataConversionWarning: A column-vector y was passed when a 1d array 

/Users/christina/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:270: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/Users/christina/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:270: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/Users/christina/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:270: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/Users/christina/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:270: DataConversionWarning: A column-vector y was passed when a 1d array 

{'predict__n_estimators': 200, 'predict__oob_score': True}
0.877519833471709
0.011553255808479596


#### Gradient Boosting

In [247]:
import xgboost

# XGB Log Sales Price
model_pipe = Pipeline(steps=[
    ('preprocess', fu),
    ('predict', XGBRegressor(objective='reg:linear'))
])

round_num = 2
model_pipe.fit(X_train, np.log(y_train).round(round_num))
print(model_pipe.score(X_dev, np.log(y_dev).round(round_num)))
print(root_mean_square_error(model_pipe.predict(X_dev), np.log(y_dev).round(round_num)))

0.9006426285450945
0.010311232750685429


In [248]:
model_pipe.fit(X_train_full, np.log(y_train_full))
xgb_pred = model_pipe.predict(X_test)
make_csv('logPrice_xgbCP5.csv', np.exp(xgb_pred))

In [242]:
np.exp(xgb_pred)

array([124404.15 , 153377.67 , 191555.95 , ..., 149376.27 , 117089.664,
       224115.48 ], dtype=float32)

### Stacking Models

In [250]:
make_csv('logPrice_ridge+XGBCP6.csv',np.exp((np.reshape(rid_pred,(1459,))+xgb_pred)/2))

array([120519.75479093, 156173.49452146, 188515.91230625, ...,
       160124.22287685, 116585.45473945, 221990.79031747])

In [246]:
rid_pred.shape

(1459, 1)

## 5. Evaluation

## 6. Deployment